In [1]:
#Preprocessing for neural network using GloVe embedding
#preprocessing packages
import pandas as pd
from nltk.stem import SnowballStemmer
import re
import glob
import spacy
import en_core_web_sm
import concurrent.futures
spacy.require_gpu()
nlp = en_core_web_sm.load()
stemmer=SnowballStemmer('english')

In [2]:
datalist=glob.glob('/home/kysha/occ_combined/*.csv')

In [3]:
combined_datalist=[pd.read_csv(i) for i in datalist] 
combined_datalist=pd.concat([pd.DataFrame(combined_datalist[i]) for i in range(len(combined_datalist))], sort=True)
rec_data=combined_datalist[['skill_category','skill_title','job_description','city','state']].fillna('None')

In [4]:
combined_datalist.columns

Index(['Unnamed: 0', 'city', 'job_description', 'job_title', 'links',
       'skill_category', 'skill_class', 'skill_title', 'state'],
      dtype='object')

In [5]:
with open ('stopwords.txt', 'r') as f:
    word=f.read().split('\n')

word=word
f.close()

In [6]:
def clean_remove_emails_https(X_data):
    X_t=[]
    for i in rec_data.job_description:
        X_t.append(' '.join([p.replace('\n',' ').replace('>','').replace('<','')\
         .replace('-',' ').lower()  for p in i.split()]))

    #remove emails and http(s) streams - can be removed
    collection=[]
    for i in X_t:
        collection.append(re.findall(r'[A-Za-z0-9_\.@+]{8,30}\.edu|[A-Za-z0-9\._@+]{8,30}\.com|\
                                     [A-Za-z0-9_\.@+]{8,30}\.gov|[A-Za-z0-9_\.@+]{8,30}\.net|\
                                     |http://[a-z0-9_\.@+/=?#]{1,150}|https://[a-z0-9_\.@+/=?#]{1,150}|\
                                     www[a-z0-9_\.@+]{8,150}|www[a-z0-9_\.@+]{8,150}', str(i)))
     
    X_emails__http_rm=[]
    for i,j in zip(X_t,collection):
        X_emails__http_rm.append(' '.join([p for p in i.split() if p not in j]))

    return X_emails__http_rm

In [7]:
%%time
X = rec_data.job_description
X_emails__http_rm_data=clean_remove_emails_https(X)

CPU times: user 34.1 s, sys: 269 ms, total: 34.4 s
Wall time: 34.7 s


In [12]:
def nouns_verbs_only(data):
    words_with_pos=[]
    for i in data:
        words_with_pos.append([[k.text, k.pos_] for k in nlp(i)])
    
    nouns_verbs_only=[]
    for i in words_with_pos:
        wrds=[k[0] for k in i]
        pos=[k[1] for k in i]
        df=pd.concat([pd.Series(wrds, name='words'),pd.Series(pos, name='pos')], axis=1)
        df=df[(df.pos=='NOUN')|(df.pos=='VERB')].reset_index().iloc[:,1:]
        nouns_verbs_only.append(df)
        
    nouns_verb_only_text=[list(i.words) for i in nouns_verbs_only]
    
       
    return nouns_verb_only_text

In [13]:
def nouns_only(data):
    words_with_pos=[]
    for i in data:
        words_with_pos.append([[k.text, k.pos_] for k in nlp(i)])
    
    nouns_only=[]
    for i in words_with_pos:
        wrds=[k[0] for k in i]
        pos=[k[1] for k in i]
        df=pd.concat([pd.Series(wrds, name='words'),pd.Series(pos, name='pos')], axis=1)
        df=df[(df.pos=='NOUN')].reset_index().iloc[:,1:]
        nouns_only.append(df)
        
    nouns_only_text=[list(i.words) for i in nouns_only]
    
       
    return nouns_only_text

In [8]:
def all_word_pos(data):
    words_with_pos=[]
    for i in data:
        words_with_pos.append([[k.text, k.pos_] for k in nlp(i)])
    
    all_pos=[]
    for i in words_with_pos:
        wrds=[k[0] for k in i]
        pos=[k[1] for k in i]
        df=pd.concat([pd.Series(wrds, name='words'),pd.Series(pos, name='pos')], axis=1)
        df=df.reset_index().iloc[:,1:]
        all_pos.append(df)
        
    #can subset df after run to focus desired positions
    return all_pos

In [ ]:
%%time
processed_text=[]
with concurrent.futures.ProcessPoolExecutor() as f:
    for i in f.map(all_word_pos, X_emails__http_rm_data):
        processed_text.append(i)
    

In [21]:
%%time 
#28 mins 56,003 rows data & Spacy GPU required (4 GPUs)
processed_text=all_word_pos(X_emails__http_rm_data)

CPU times: user 28min 25s, sys: 2.93 s, total: 28min 28s
Wall time: 28min 28s


In [31]:
rec_data.head()

,skill_category,skill_title,job_description,city,state
0,Mathematics.xls,Mathematicians,Be a LEADer. Teach Math at LEAD Public Schools...,Nashville,TN
1,Mathematics.xls,Mathematicians,Duties \n Summary \n About the Position: This...,None,None
2,Mathematics.xls,Mathematicians,K Health is transforming the world of medicine...,New York,NY
3,Mathematics.xls,Mathematicians,The Lamont-Doherty Earth Observatory in Palisa...,New York,NY
4,Mathematics.xls,Mathematicians,Senior Business Analyst / Lead Business Analys...,New York,NY


In [55]:
words=[]
pos=[]
for i in processed_text:
    words.append(i.words)
    pos.append(i.pos)

In [63]:
pos_dict=[]
for i,k in zip(pos,words):
    pos_dict.append(dict(zip(list(k),list(i))))

In [65]:
rec_data['rec_data']=pd.Series(pos_dict)

In [79]:
from datetime import date

In [80]:
rec_data.to_csv(f'processed_{date.today()}.csv')

In [16]:
def rm_stopword_stem_word_cnt(data):
       
    no_stopwords=[]
    for i in data:
        no_stopwords.append(' '.join([j for j in i if j not in word and len(j) >=3]))
        
    '''no_stop_stemmed_words=[]
    for p in no_stopwords:
        no_stop_stemmed_words.append(' '.join([stemmer.stem(k) for k in p.split()]))'''
    
    data_len=[]
    for i in no_stopwords:
        data_len.append(len(i.split()))
        
    return no_stop_stemmed_words,data_len

In [17]:
%%time
no_stop_stemmed=rm_stopword_stem_word_cnt(nouns_verb_only_text)

CPU times: user 2min 35s, sys: 12 ms, total: 2min 35s
Wall time: 2min 35s


In [18]:
rec_data['no_stopwords_stemmed']=pd.Series(no_stop_stemmed[0])
rec_data['data_len']=pd.Series(no_stop_stemmed[1])
rec_data=rec_data.fillna('None')
data=rec_data[rec_data.skill_category!='None'].copy()
data2=data[(data.data_len>0)]
data3=data2[data2.data_len>data2.data_len.mean()].reset_index().iloc[:,1:]

/home/kysha/anaconda3/envs/kyshaenv2/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/kysha/anaconda3/envs/kyshaenv2/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
data3.head()

,skill_category,skill_title,job_description,city,state,no_stopwords_stemmed,data_len
0,Mathematics.xls,Mathematicians,Duties \n Summary \n About the Position: This...,None,None,duti posit posit duti locat applic level targe...,796
1,Mathematics.xls,Mathematicians,Senior Business Analyst / Lead Business Analys...,New York,NY,busi analyst busi analyst organ group chegg gr...,309
2,Mathematics.xls,Mathematicians,"Manager, Business Analytics \nAnalytics | New ...",New York,NY,manag organ group chegg group data decis chegg...,238
3,Mathematics.xls,Mathematicians,About Mastery: \nWe believe educational inequi...,Camden,NJ,masteri inequ problem countri right issu day m...,211
4,Mathematics.xls,Mathematicians,About Mastery: \nWe believe educational inequi...,Philadelphia,PA,masteri inequ problem countri right issu day m...,211


In [20]:
data3.to_csv('occ_preprocessed_.csv')